# Qredo tokenomics simulation - parameter sweep

### June 2023

## 1. Imports

In [ ]:
import os
import pickle
import timeit
import pandas as pd
import numpy as np
import itertools

import warnings
warnings.filterwarnings('ignore')

In [ ]:
code_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))

import sys
sys.path.append(code_dir)

from mechaqredo.sim import run_param_sweep_sim

## 2. Load scenarios

In [ ]:
forecast_length = 365*2
outfolder = os.path.realpath("../data/sim_data")
scenario_file = os.path.join(outfolder, f"params_scenarios.pickle")
with open(scenario_file, "rb") as fp:
    scenario_dict_list = pickle.load(fp)
    
scenario_dict_list[0].keys()

## 3. Set ranges for tokenomic params

In [ ]:
file = os.path.realpath("../data/balances.csv")
wallet_df = pd.read_csv(file)
wallet_df.balance.quantile([0.01, 0.05, 0.1])

**Initial ranges:**

```python
param_ranges_dict = {
    "tipping_rate": [0.05, 0.2, 0.4],
    "protocol_fee_rate": [0.001,0.01, 0.1, 1],
    "min_stake_amount": [1, 160, 5000, 12000],
    "min_stake_duration": [7*2, 7*4, 7*8],
    "validator_reward_share": list(np.arange(0.1, 1, 0.1)),
    "staking_rewards_vesting_decay_rate": [np.log(2) / (i * 365) for i in [3.0, 4.0, 5.0]],
    "release_rate_a": [1/3, 1/2, 1.0],
    "release_rate_b": [0.5, 1.0, 2.0],
}
```

**Current ranges:**

In [ ]:
param_ranges_dict = {
    "tipping_rate": [0.05, 0.2],
    "protocol_fee_rate": [0.001, 0.1],
    "min_stake_amount": [200, 5000],
    "min_stake_duration": [7*4, 7*8],
    "validator_reward_share": list(np.arange(0.1, 1, 0.3)),
    "staking_rewards_vesting_decay_rate": [np.log(2) / (i * 365) for i in [3.0, 4.0]],
    "release_rate_a": [1/2, 1.0],
    "release_rate_b": [0.5, 1.0],
}

**Time estimation per scenario (minutes):**

In [ ]:
a = len(list(itertools.product(*param_ranges_dict.values())))
b = len(scenario_dict_list[0]["data_dict_list"])
single_sim = 0.011927916999999955

(single_sim*a*b)/60

## 3. Run parameter sweep

In [ ]:
sim=1
for scenario_dict in scenario_dict_list:
    params_dict = scenario_dict["params_dict"]
    data_dict_list = scenario_dict["data_dict_list"]
    sweep_df = run_param_sweep_sim(
        forecast_length,
        params_dict,
        param_ranges_dict,
        data_dict_list=data_dict_list,
    )
    sweep_df['token_scenario'] = scenario_dict['token_scenario']
    sweep_df['usage_scenario'] = scenario_dict['usage_scenario']
    sweep_df['staking_scenario'] = scenario_dict['staking_scenario']
    stop = timeit.default_timer()
    file_name = f"sim_df_scenario_{sim}.csv"
    scenario_file = os.path.join(outfolder, file_name)
    sweep_df.to_csv(scenario_file, index=False)
    sim+=1